In this notebook, we will calculate features numerically (without trotterization).

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
# Add the fourier_learning_ibm package to the path
# This is necessary only when running the notebook in the docker container
import sys, pprint

sys.path.append("/home/jovyan/fourier_learning_ibm/")
pprint.pprint(sys.path)

# This is necessary only when running the notebook in the docker container
%cd fourier_learning_ibm/

['/home/jovyan',
 '/opt/conda/lib/python311.zip',
 '/opt/conda/lib/python3.11',
 '/opt/conda/lib/python3.11/lib-dynload',
 '',
 '/opt/conda/lib/python3.11/site-packages',
 '/home/jovyan/fourier_learning_ibm/']
/home/jovyan/fourier_learning_ibm


In [3]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import pandas as pd
import os
import time
from heisenberg import (
    HeisenbergModel,
    get_graph,
    get_positions,
    # get_initial_layout,
    # get_prob0,
)
from setup import setup_backend
from qiskit import transpile
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler, Batch
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel
import pickle
import networkx as nx
from datetime import datetime, timezone
import json
import math

In [4]:
import warnings

warnings.filterwarnings("ignore")

### Config backend

In [5]:
backend_qpu, backend_sim_noiseless, backend_sim_noisy = setup_backend(
    qpu_name="ibm_marrakesh"
)

Using backend QPU: <IBMBackend('ibm_marrakesh')>
Using backend noiseless simulator: AerSimulator('aer_simulator')

NoiseModel:
  Basis gates: ['cz', 'delay', 'for_loop', 'id', 'if_else', 'measure', 'reset', 'rz', 'switch_case', 'sx', 'x']
  Instructions with noise: ['reset', 'measure', 'id', 'x', 'cz', 'sx']
  Qubits with noise: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155

### Prepare parameters and load dataset

In [6]:
n_qubits = 4
# C = 3
# C = 3 * (n_qubits - 1)
C = 5
threshold = 0.6

# Load dataset (orient="records")
df_n_step = pd.read_json(
    f"results/get_param/{n_qubits}Q/threshold{threshold:.2f}/optimal_n_step.json",
    orient="records",
)
n_samples = 55

# Note that K = n_features-1.
# n_features counts only the number of real features, also the one of t=0, i.e., constant term.
# The number of all features, including imaginary numbers, is 2K+1.
n_features = len(df_n_step)

times = df_n_step["t"].values

display(df_n_step)
print(f"n_qubits: {n_qubits}")
print(f"C: {C}")
print(f"threshold: {threshold}")
print(f"n_samples: {n_samples}")
print(f"n_features: {n_features}")
print(f"times: {times}")

# Load dataset (orient="records")
df_dataset = pd.read_json(
    f"results/get_dataset/{n_qubits}Q/dataset.json", orient="records"
)
all_Js = df_dataset["Js"].values
all_expected_values = df_dataset["expected_value"].values
n_samples = len(all_Js)
display(df_dataset)

,t,n_step,sta_fidelity
0,0.000000,1,1.000000
1,0.628319,1,0.960905
2,1.256637,1,0.934116
3,1.884956,2,0.923847
4,2.513274,2,0.899309
5,3.141593,3,0.878686
6,3.769911,3,0.862589
7,4.398230,4,0.840110
8,5.026548,5,0.813450
9,5.654867,5,0.800024


n_qubits: 4
C: 5
threshold: 0.6
n_samples: 55
n_features: 19
times: [ 0.          0.62831853  1.25663706  1.88495559  2.51327412  3.14159265
  3.76991118  4.39822972  5.02654825  5.65486678  6.28318531  6.91150384
  7.53982237  8.1681409   8.79645943  9.42477796 10.05309649 10.68141502
 11.30973355]


,sample,Js,expected_value,expected_value_diag,diff
0,0,"[0.3949339961, -0.0881126084, 0.5169533955]",5.547457,5.547450,6.426300e-06
1,1,"[0.1829492443, -0.3761751478, 0.4408756079]",3.841905,3.841881,2.462770e-05
2,2,"[0.2841284266, 0.3112471996, -0.4046243738]",1.129636,1.129639,3.412100e-06
3,3,"[-0.0819280326, -0.2133520868, 0.7047198806]",4.731457,4.731416,4.172410e-05
4,4,"[0.2749649723, 0.616884329, -0.1081506987]",0.852425,0.852410,1.459610e-05
5,5,"[-0.3572377034, 0.0714901473, -0.5712721494]",0.828226,0.828227,1.070800e-06
6,6,"[0.4567030859, 0.1835342376, 0.3597626765]",3.697788,3.697784,4.380700e-06
7,7,"[-0.1441345638, 0.4663640207, 0.3895014155]",1.258985,1.258995,1.064270e-05
8,8,"[0.45117141930000004, -0.49489387500000004, -0...",3.279570,3.279566,4.387400e-06
9,9,"[-0.4631642178, -0.3509909223, 0.1858448599000...",1.577586,1.577586,7.130000e-08


# Calculate Fourier feature

## Exact (No Trotterization)
Exact means that we compute the matrix exponential of the Hamiltonian, not Trotterized.

In [7]:
data = []
probs_phase0_exact = {}
probs_phase1_exact = {}
probs_phase2_exact = {}
probs_phase3_exact = {}

# 保存用のファイルを初期化
with open(f"results/fourier_feature_exact/temp_progress.txt", "w") as f:
    f.write("")  # ファイルを空にする

for i in range(n_samples):
    # 途中経過を表示 + ファイルに保存
    progress_report = f"Calculating features for sample {i}/{n_samples}"
    print(progress_report)
    with open(f"results/fourier_feature_exact/temp_progress.txt", "a") as f:
        f.write(progress_report + "\n")

    Js = all_Js[i]
    G = get_graph(n_qubits, Js)
    heisenberg = HeisenbergModel(n_qubits, G)

    lambda_ref = np.sum(Js)  # Reference eigenvalue.
    features_exact = []

    probs_phase0_exact[f"sample{i}"] = {}
    probs_phase1_exact[f"sample{i}"] = {}
    probs_phase2_exact[f"sample{i}"] = {}
    probs_phase3_exact[f"sample{i}"] = {}

    # Compute the Fourier features for different times
    for k in range(n_features):
        start = time.time()
        final_state_phase0, _ = heisenberg.exact_simulation(times[k], phase=0)
        final_state_phase1, _ = heisenberg.exact_simulation(times[k], phase=1)
        final_state_phase2, _ = heisenberg.exact_simulation(times[k], phase=2)
        final_state_phase3, _ = heisenberg.exact_simulation(times[k], phase=3)

        # "00...00" がなければ確率 0 を取得
        prob_phase0 = final_state_phase0.probabilities_dict().get("0" * n_qubits, 0)
        prob_phase1 = final_state_phase1.probabilities_dict().get("0" * n_qubits, 0)
        prob_phase2 = final_state_phase2.probabilities_dict().get("0" * n_qubits, 0)
        prob_phase3 = final_state_phase3.probabilities_dict().get("0" * n_qubits, 0)

        probs_phase0_exact[f"sample{i}"][f"f_{k}"] = prob_phase0
        probs_phase1_exact[f"sample{i}"][f"f_{k}"] = prob_phase1
        probs_phase2_exact[f"sample{i}"][f"f_{k}"] = prob_phase2
        probs_phase3_exact[f"sample{i}"][f"f_{k}"] = prob_phase3

        inner_product = np.exp(-1j * lambda_ref * times[k]) * (
            (prob_phase0 - prob_phase1) + 1j * (prob_phase2 - prob_phase3)
        )

        features_exact.append(inner_product.real)
        if k != 0:
            features_exact.append(inner_product.imag)
        end = time.time()
        elapsed_time = end - start

        # 途中経過を表示 + ファイルに保存
        progress_report = (
            f"Time: {times[k]:.3f}, Feature: {k}, Elapsed time: {elapsed_time:.2f}[s]"
        )
        print(progress_report)
        with open(f"results/fourier_feature_exact/temp_progress.txt", "a") as f:
            f.write(progress_report + "\n")
    data.append([i, *features_exact, all_expected_values[i]])

# Create column names for the DataFrame
columns = []
columns.append("sample_id")
for k in range(n_features):
    columns.append(f"f_{k} Re")
    if k != 0:
        columns.append(f"f_{k} Im")
columns.append("expected_value")

# Convert to a DataFrame
df_exact = pd.DataFrame(data, columns=columns)
display(df_exact)

# Save the features
path = f"results/fourier_feature_exact/{n_qubits}Q"
df_exact.to_json(f"{path}/features.json", orient="records", indent=4)

# Save the probabilities, for reference
with open(f"{path}/probs_phase0.json", "w") as f:
    json.dump(probs_phase0_exact, f)

with open(f"{path}/probs_phase1.json", "w") as f:
    json.dump(probs_phase1_exact, f)

with open(f"{path}/probs_phase2.json", "w") as f:
    json.dump(probs_phase2_exact, f)

with open(f"{path}/probs_phase3.json", "w") as f:
    json.dump(probs_phase3_exact, f)

Calculating features for sample 0/55
Time: 0.000, Feature: 0, Elapsed time: 0.01[s]
Time: 0.628, Feature: 1, Elapsed time: 0.02[s]
Time: 1.257, Feature: 2, Elapsed time: 0.02[s]
Time: 1.885, Feature: 3, Elapsed time: 0.02[s]
Time: 2.513, Feature: 4, Elapsed time: 0.02[s]
Time: 3.142, Feature: 5, Elapsed time: 0.02[s]
Time: 3.770, Feature: 6, Elapsed time: 0.02[s]
Time: 4.398, Feature: 7, Elapsed time: 0.02[s]
Time: 5.027, Feature: 8, Elapsed time: 0.02[s]
Time: 5.655, Feature: 9, Elapsed time: 0.02[s]
Time: 6.283, Feature: 10, Elapsed time: 0.02[s]
Time: 6.912, Feature: 11, Elapsed time: 0.02[s]
Time: 7.540, Feature: 12, Elapsed time: 0.02[s]
Time: 8.168, Feature: 13, Elapsed time: 0.02[s]
Time: 8.796, Feature: 14, Elapsed time: 0.02[s]
Time: 9.425, Feature: 15, Elapsed time: 0.02[s]
Time: 10.053, Feature: 16, Elapsed time: 0.02[s]
Time: 10.681, Feature: 17, Elapsed time: 0.02[s]
Time: 11.310, Feature: 18, Elapsed time: 0.02[s]
Calculating features for sample 1/55
Time: 0.000, Feature:

,sample_id,f_0 Re,f_1 Re,f_1 Im,f_2 Re,f_2 Im,f_3 Re,f_3 Im,f_4 Re,f_4 Im,...,f_14 Im,f_15 Re,f_15 Im,f_16 Re,f_16 Im,f_17 Re,f_17 Im,f_18 Re,f_18 Im,expected_value
0,0,1.0,0.560687,0.420610,-0.004403,0.161817,-0.059900,-0.039643,-0.271602,0.206875,...,-0.678152,0.369094,-0.270008,0.133341,-0.063803,0.217882,-0.139135,0.444786,0.274679,5.547457
1,1,1.0,0.659937,0.510626,0.010218,0.488936,-0.286929,0.028484,-0.046904,-0.320783,...,-0.657615,0.395262,-0.163542,0.060216,0.399626,-0.553024,0.448836,-0.809490,0.015124,3.841905
2,2,1.0,0.786771,-0.242274,0.321462,-0.343534,-0.029329,-0.292498,-0.033683,-0.209428,...,0.014087,-0.156450,0.108880,-0.219413,0.192764,0.020689,0.092465,0.344706,-0.210200,1.129636
3,3,1.0,0.532348,0.345389,-0.228494,-0.036971,-0.310471,-0.712980,0.189510,-0.733352,...,0.241268,0.023086,0.325295,0.139495,0.221252,0.259271,0.150223,0.165745,0.164318,4.731457
4,4,1.0,0.896922,-0.276852,0.648533,-0.508849,0.381886,-0.626635,0.188313,-0.578558,...,-0.378536,-0.247978,-0.220286,-0.212152,-0.104582,-0.187853,-0.093106,-0.173653,-0.155228,0.852425
5,5,1.0,0.543823,-0.406658,-0.018078,-0.098544,-0.025449,0.128316,-0.177597,-0.120177,...,0.654129,0.290107,0.000532,-0.197580,-0.077482,-0.136399,0.146674,0.060063,-0.061320,0.828226
6,6,1.0,0.705606,0.276342,0.271425,0.124807,0.193243,-0.033918,0.131369,0.233588,...,-0.426300,0.023777,-0.520185,0.489157,-0.125354,0.519695,0.347793,0.409986,0.349977,3.697788
7,7,1.0,0.863673,-0.145024,0.554199,-0.304717,0.279728,-0.416050,0.178488,-0.375409,...,-0.012776,0.447312,-0.063669,0.615878,-0.140069,0.636338,-0.244339,0.468020,-0.390714,1.258985
8,8,1.0,0.705002,0.477656,0.083901,0.546686,-0.311094,0.156715,-0.137908,-0.343958,...,0.381574,0.054260,0.421317,-0.114499,0.174185,0.040231,-0.117316,0.320835,-0.210460,3.279570
9,9,1.0,0.816539,0.064271,0.414581,0.198627,0.098563,0.337863,0.052650,0.328533,...,-0.277918,0.662827,-0.109879,0.945172,0.006338,0.903950,0.063562,0.546511,0.149703,1.577586
